In [ ]:
############################# needed for creating strategy
import os
import sys
sys.path.append("agent/util")
from simulation import BridgeAgent
from strategy import Strategy,StrategyActionSolver,MainSubStrategy,RandomMixStrategy

############################# needed for evaluating a strategy
import pprint
import itertools

from tqdm.notebook import tqdm

from simulation import Simulator,FixedActionSolver,exportJsonForVisualization,logger as simulation_logger
from visualizer import displayRoadNetStatistics

############################# log level setting
import logging
logging.basicConfig(level=logging.ERROR)
simulation_logger.setLevel(logging.INFO)

#############################
sys.path.append("agent")
from CBEngine_round3 import CBEngine_round3 as CBEngine_rllib_class

class SimulatorRunner:
    def __init__(self,
            simulator_cfg_file,
            metric_period,
            exportAgentLog=False,
            earlyStoppingDelayIndex=1.6,
        ):
        sim=Simulator(CBEngine_rllib_class,simulator_cfg_file,metric_period)

        print("****** simulator_configs ******")
        pprint.pprint(sim.simulator_configs)
        print("****** roadNet summary ******")
        sim.roadNet.printSummary()
        print("****** roadNet statistics ******")
        displayRoadNetStatistics(sim.roadNet)
        
        self.sim=sim
        self.exportAgentLogPath=sim.simulator_configs['report_log_addr']+"/agent.txt" if exportAgentLog else None
        self.earlyStoppingDelayIndex=earlyStoppingDelayIndex
        
        self.total_served_vehicles_dic={}
        self.delay_index_dic={}
        self.lastTime_dic={}

    def runActionSolver(self,
            actionSolver,
            exportActionLogPath=None,
            recordKey=None
        ):
        print("****** evaluation ******")
        total_served_vehicles,delay_index,lastTime=self.sim.evaluate(
            BridgeAgent(actionSolver),
            earlyStoppingDelayIndex=self.earlyStoppingDelayIndex,
            tqdm=tqdm
        )
        if recordKey is not None:
            self.total_served_vehicles_dic[recordKey]=total_served_vehicles
            self.delay_index_dic[recordKey]=delay_index
            self.lastTime_dic[recordKey]=lastTime
        
        if exportActionLogPath is not None:
            actionSolver.exportActionLog(exportActionLogPath)
            print("action log exported:",exportActionLogPath)
            
    def runStrategy(self,
            strategy,
            actionTestCount,
            exportActionLogPath=None,
            recordKey=None
        ):
        actionSolver = StrategyActionSolver(
            strategy,
            export_agent_log_path=self.exportAgentLogPath,
            action_test_count=actionTestCount
        )
        self.runActionSolver(actionSolver,exportActionLogPath,recordKey)
            
    def printAllEvalRecord(self):
        print("****** all evaluation record ******")
        pprint.pprint(self.total_served_vehicles_dic)
        pprint.pprint(self.delay_index_dic)
        pprint.pprint(self.lastTime_dic)
        

In [ ]:
##################################### 混合strategyのテストを実行します
###parameters for simulator
simulator_cfg_file = "cfg/simulator_round3_flow0.cfg"
metric_period = 200
earlyStoppingDelayIndex=1.6

###parameters for DNN agent
dnnCheckpointFileName="round2_18_18_5_r1.ckpt"
dnnNumSegmentInbound=18
dnnNumSegmentOutbound=18
dimVehiclesVec=64
unitsDict={
    'inbound':[64,64,64,64],
}

###parameters for rule base agent
timeThresForCalcReward=10.2
prohibitDecreasingGoalDistance=True
prohibitDecreasingSpeed=True

###parameters for mixing strategy 
dnnStrategyRatio=0.6
ruleStrategyRatio=0.1

###parameters for actionSolver
actionTestCount=1
exportAgentLog=True

###parameters for visualization
exportForVisualization=False

#####################################
runner=SimulatorRunner(
    simulator_cfg_file,
    metric_period,
    exportAgentLog,
    earlyStoppingDelayIndex,
)

# dnnStrategy=Strategy.createStrategy(
#     'dnn_run_distance',{
#         'checkpointPath':"ckpt/"+dnnCheckpointFileName,
#         'numSegmentInbound':dnnNumSegmentInbound,
#         'numSegmentOutbound':dnnNumSegmentOutbound,
#         'unitsDict':unitsDict,
#         'dimVehiclesVec':dimVehiclesVec,
#     }
# )
dnnStrategy=Strategy.createStrategy(
    'dnn_saved_run_distance',{
        'savedModelPath':"saved_model/"+os.path.splitext(dnnCheckpointFileName)[0],
        'numSegmentInbound':dnnNumSegmentInbound,
        'numSegmentOutbound':dnnNumSegmentOutbound,
    }
)

ruleStrategy=Strategy.createStrategy(
    'run_distance',{
        'timeThresForCalcReward':timeThresForCalcReward,
        'prohibitDecreasingGoalDistance':prohibitDecreasingGoalDistance,
        'prohibitDecreasingSpeed':prohibitDecreasingSpeed,
    }
)

if dnnStrategyRatio is None or dnnStrategyRatio==0:
    strategy=ruleStrategy
elif ruleStrategyRatio is None or ruleStrategyRatio==0:
    strategy=dnnStrategy
else:
    strategy=RandomMixStrategy({
        dnnStrategy:dnnStrategyRatio,
        ruleStrategy:ruleStrategyRatio,
    })
    
runner.runStrategy(
    strategy,
    actionTestCount,
    exportActionLogPath = "action_log/{}_dnn{}_rule{}_{}.json".format(
        dnnCheckpointFileName.split(".")[0],
        dnnStrategyRatio,
        ruleStrategyRatio,
        actionTestCount,
    )
)

if exportForVisualization:
    print("now exporting simulation output as json for visualization")
    exportJsonForVisualization(simulator_cfg_file,every_n=10,tqdm=tqdm)

In [ ]:
# routeを使用してるのでROUND3では動かない！！！！！！！

# ##################################### rule base strategyのテストを実行します（パラメータ探索機能付き）
# ###parameters for simulator
# simulator_cfg_file = "cfg/simulator_round2.cfg"
# metric_period = 200
# earlyStoppingDelayIndex=1.6

# ###parameters for rule base agent
# maxDepth_list=[4]
# timeThresForCalcReward_list=[10.2]
# prohibitDecreasingGoalDistance=True
# prohibitDecreasingSpeed=True

# ###parameters for actionSolver
# actionTestCount_list=[1]
# exportAgentLog=True

# ###parameters for visualization
# exportForVisualization=False

# #####################################
# runner=SimulatorRunner(
#     simulator_cfg_file,
#     metric_period,
#     exportAgentLog,
#     earlyStoppingDelayIndex,
# )

# params_list=list(itertools.product(
#     maxDepth_list,
#     timeThresForCalcReward_list,
#     actionTestCount_list,
# ))

# for params in params_list:
#     print(params)
#     strategy=Strategy.createStrategy(
#         'run_distance',
#         {
#             'maxDepth':params[0],
#             'timeThresForCalcReward':params[1],
#             'prohibitDecreasingGoalDistance':prohibitDecreasingGoalDistance,
#             'prohibitDecreasingSpeed':prohibitDecreasingSpeed,
#         }
#     )
#     runner.runStrategy(strategy,params[2],recordKey=params)

# runner.printAllEvalRecord()    

# if exportForVisualization:
#     print("now exporting simulation output as json for visualization")
#     exportJsonForVisualization(simulator_cfg_file,every_n=10,tqdm=tqdm)

In [ ]:
# #####################################
# ###parameters for simulator
# metric_period = 1
# simulator_cfg_file = "cfg/simulator_test2.cfg"

# ###parameters for agent
# timeToActionsDict={
#     0:{
#         0:5,
#         1:7,
#     },
#     30:{
#         0:6,
#         1:8,
#     },
#     40:{
#         0:8,
#         1:8,
#     },
#     70:{
#         0:4,
#         1:6,
#     },
#     90:{
#         1:8,
#     },
# }
# #####################################
# runner=SimulatorRunner(
#     simulator_cfg_file,
#     metric_period,
# )

# runner.runActionSolver(FixedActionSolver(timeToActionsDict))

# exportJsonForVisualization(simulator_cfg_file,tqdm=tqdm)